In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as mpl

%matplotlib inline
mpl.style.use('ggplot')
mpl.rcParams['figure.figsize'] = 16,6


# 15.1.a

A portfolio manager intends to launch a strategy that targets an annualized SR of 2. Bets have a precision rate of 60%, with weekly frequency. The exit conditions are 2% for profit taking and -2% for stop-loss.

Is this strategy viable?

In [3]:
def run_sr_trials(p, pt=1, sl=1, trials=100000):
    out = []
    for i in range(trials):
        rnd = np.random.binomial(n=1, p=p)
        x = (pt if rnd == 1 else -sl)
        out.append(x)
    return np.mean(out) / np.std(out)

In [32]:
p = 0.6
freq = 52
sr = run_sr_trials(p, trials=100000)
print("As stated the strategy is expected to achieve an annualized SR of ~{:.3f}.".format(float(np.sqrt(freq) * sr)))

As stated the strategy is expected to achieve an annualized SR of ~1.488.


# 15.1.b

A portfolio manager intends to launch a strategy that targets an annualized SR of 2. Bets have a precision rate of 60%, with weekly frequency. The exit conditions are 2% for profit taking and -2% for stop-loss.

*Ceteris paribus*, what is the required precision rate that would make the strategy profitable?

In [12]:
for p_ in np.linspace(0.55, 1.0, 50):
    sr = run_sr_trials(p_, trials=100000)
    if np.sqrt(freq) * sr > 2:
        break
         
print("Achieving an annualized SR of 2 while keeping the other values the same would require a precision rate of {:.3f}.".format(p_))


Achieving an annualized SR of 2 while keeping the other values the same would require a precision rate of 0.642.


# 15.1.c

A portfolio manager intends to launch a strategy that targets an annualized SR of 2. Bets have a precision rate of 60%, with weekly frequency. The exit conditions are 2% for profit taking and -2% for stop-loss.

For what betting frequency is the target achievable?

In [16]:
sr = run_sr_trials(p, trials=1000000)
target_sr = 2
freq_ = (target_sr / sr) ** 2
print("Achieving an annualized SR of 2 while keeping the other values the same would require a betting frequency of {:.0f} trades per year.".format(freq_))


Achieving an annualized SR of 2 while keeping the other values the same would require a betting frequency of 96 trades per year.


# 15.1.d

A portfolio manager intends to launch a strategy that targets an annualized SR of 2. Bets have a precision rate of 60%, with weekly frequency. The exit conditions are 2% for profit taking and -2% for stop-loss.

For what profit-taking threshold is the target achievable?

In [19]:
pt = 2
sl = 2
for pt_ in np.linspace(2, 4, 200):
    sr = run_sr_trials(p, pt_, sl, 100000)
    if np.sqrt(freq) * sr > 2:
        break
         
print("Achieving an annualized SR of 2 while keeping the other values the same would require a profit take limit of {:.2f}%.".format(pt_))


Achieving an annualized SR of 2 while keeping the other values the same would require a profit take limit of 2.30%.


# 15.1.e

A portfolio manager intends to launch a strategy that targets an annualized SR of 2. Bets have a precision rate of 60%, with weekly frequency. The exit conditions are 2% for profit taking and -2% for stop-loss.

What would be an alternative stop-loss?

In [20]:
for sl_ in np.linspace(2, 1, 200):
    sr = run_sr_trials(p, pt, sl_, 100000)
    if np.sqrt(freq) * sr > 2:
        break
         
print("Achieving an annualized SR of 2 while keeping the other values the same would require a stop loss limit of {:.2f}%.".format(sl_))


Achieving an annualized SR of 2 while keeping the other values the same would require a stop loss limit of 1.74%.


# 15.2.a

Following up on the strategy from exercise 1.

What is the sensitivity of SR to a 1% change in each parameter?

In [11]:
p = 0.55
pt = 2
sl = 2
freq = 52

def jiggle(v):
    return [v * 0.99, v, v * 1.01]

print('freq', pd.Series([np.sqrt(freq_) * run_sr_trials(p, pt, sl, 10000000) for freq_ in jiggle(freq)]).pct_change().std())
print('p   ', pd.Series([np.sqrt(freq) * run_sr_trials(p_, pt, sl, 10000000) for p_ in jiggle(p)]).pct_change().std())
print('pt  ', pd.Series([np.sqrt(freq) * run_sr_trials(p, pt_, sl, 10000000) for pt_ in jiggle(pt)]).pct_change().std())
print('sl  ', pd.Series([np.sqrt(freq) * run_sr_trials(p, pt, sl_, 10000000) for sl_ in jiggle(sl)]).pct_change().std())



freq 0.003589926917184778
p    0.02066441077148859
pt   0.007574011138762322
sl   0.006222697554722991


In [144]:
print("The SR is most sensitive to changes in precision.")

The SR is most sensitive to changes in precision


# 15.2.b

Following up on the strategy from exercise 1.

Given these sensitivies, and assuming that all parameters are equally hard to improve, which one offers the lowest hanging fruit?

**A: Under these assumptions -- precision.**

# 15.2.c

Following up on the strategy from exercise 1.

Does changing any of the parameters in exercise 1 impact the others? For example, does changing the betting frequency modify the precision rate, etc.?

**A: In this experiment the parameters function independent of one another. In real life, changing profit-take & stop-loss parameters would very likely impact precision and betting frequency.**

# 15.3.a

Suppose a strategy that generates monthly bets over two years, with returns following a mixture of two Gaussians distributions. The first distribution has a mean of -0.1 and a standard deviation of 0.12. The second distribution has a mean of 0.06 and a standard deviation of 0.03. The probability that a draw comes from the first distribution is 0.15.

Following Lopez de Prado and Peijan [2004] and Lopez de Prado and Foreman [2014], derive the first 4 moments for the mixture's returns.

# 15.3.b

Suppose a strategy that generates monthly bets over two years, with returns following a mixture of two Gaussians distributions. The first distribution has a mean of -0.1 and a standard deviation of 0.12. The second distribution has a mean of 0.06 and a standard deviation of 0.03. The probability that a draw comes from the first distribution is 0.15.

What is the annualized SR?

In [22]:
import scipy.stats as ss

def binHR(sl, pt, freq, tSR):
    '''
    Given a trading rule characterized by the parameters {sl, pt, freq},
    what's the min precision p required to achieve a Sharpe ratio tSR?
    1) Inputs
    sl: stop loss threshold
    pt: profit taking threshold
    freq: number of bets per year
    tSR: target annual Sharpe ratio
    2) Output
    p: the min precision rate p required to achieve tSR
    '''
    a = (freq + tSR ** 2) * (pt - sl) ** 2
    b = (2 * freq * sl - tSR ** 2 * (pt - sl)) * (pt - sl)
    c = freq * sl ** 2
    p = (-b + (b ** 2 - 4 * a * c) ** 0.5) / (2.0 * a)
    return p

def binFreq(sl, pt, p, tSR):
    ''' 
    Given a trading rule characterized by the parameters {sl, pt, freq}, what's the number
    of bets/year needed to achieve a Sharpe ratio tSR with precision rate p?
    Note: Equation with radicals, check for extraneous solution. 
    1) Inputs
    sl: stop loss threshold
    pt: profit taking threshold
    p: precision rate
    tSR: target annual Sharpe ratio
    2) Output
    freq: number of bets per year needed
    '''
    freq = (tSR * (pt - sl)) ** 2 * p * (1 - p) / ((pt - sl) * p + sl) ** 2 # possible extraneous
    if not np.isclose(binSR(sl, pt, freq, p), tSR):
        return
    return freq

def mixGaussians(mu1, mu2, sigma1, sigma2, prob1, nObs):
    # Random draws from a mixture of gaussians
    ret1 = np.random.normal(mu1, sigma1, size=int(nObs * prob1))
    ret2 = np.random.normal(mu2, sigma2, size=int(nObs) - ret1.shape[0])
    ret = np.append(ret1, ret2, axis=0)
    np.random.shuffle(ret)
    return ret

def probFailure(ret, freq, tSR):
    # Derive probability that the strategy may fail
    rPos, rNeg = ret[ret > 0].mean(), ret[ret <= 0].mean()
    p = ret[ret > 0].shape[0] / float(ret.shape[0])
    thresP = binHR(rNeg, rPos, freq, tSR)
    risk = ss.norm.cdf(thresP, p, p * (1 - p))
    return risk

mu1, mu2, sigma1, sigma2, prob1, nObs = -0.1, 0.06, 0.12, 0.03, 0.15, 100000
ret = mixGaussians(mu1, mu2, sigma1, sigma2, prob1, nObs)


In [23]:
sr = np.mean(ret) / np.std(ret)
asr = np.sqrt(12) * sr
print("The annualized SR is {:.2f}.".format(asr))

The annualized SR is 1.58


# 15.3.c

Suppose a strategy that generates monthly bets over two years, with returns following a mixture of two Gaussians distributions. The first distribution has a mean of -0.1 and a standard deviation of 0.12. The second distribution has a mean of 0.06 and a standard deviation of 0.03. The probability that a draw comes from the first distribution is 0.15.

Using these moments, compute PSR[1] (see Chapter 14). At a 95% confidence interval, would you discard this strategy?

In [33]:
from stats import psr
ret = pd.Series(ret)
print("The PSR is {}.".format(psr(sr, len(ret), ret.skew(), ret.kurtosis())))

The PSR is 1.0.


# 15.4

Using Snippet 15.5, compute $P[p \lt p_{\theta*=1}]$ for the strategy described in exercise 3. At a significance level of 0.05, would you discard this strategy? Is this result consistent with $PSR[\theta*]$?

In [31]:
# 1) Parameters
mu1, mu2, sigma1, sigma2, prob1, nObs = -0.1, 0.06, 0.12, 0.03, 0.15, 100000
tSR, freq = asr, 12
# 2) Generate sample from mixture
ret = mixGaussians(mu1, mu2, sigma1, sigma2, prob1, nObs)
# 3) Compute prob failure
probF = probFailure(ret, freq, tSR)
print("The strategy is estimated to fail {:.2%} of the time.".format(probF))


The strategy is estimated to fail 41.09% of the time.


# 15.5

In general what result do you expect to be more accurate, $PSR[\theta*]$ or $P[p \lt p_{\theta*=1}]$? How are these two methods complementary?

# 15.6.a

Re-examine the results from Chapter 13, in light of what you have learned in this chapter. 

Does the asymmetry between profit taking and stop-loss thresholds in OTRs make sense?

# 15.6.b

Re-examine the results from Chapter 13, in light of what you have learned in this chapter. 

What is the range of $p$ implied by Figure 13.1, for a daily betting frequency?

# 15.6.c

Re-examine the results from Chapter 13, in light of what you have learned in this chapter. 

What is the range of $p$ implied by Figure 13.5, for a weekly betting frequency?